In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

df = None
for path in Path('data/by_file/token_type-ratio').iterdir():  
    dfi = pd.read_csv(path)
    dfi = dfi.set_index('num')
    dfi.columns = pd.MultiIndex.from_tuples([ (col, path.stem ) for col in dfi.columns ])
    
    if df is None:
        df = dfi
    else:
        df = df.join(dfi, how='outer')

df = df.fillna(0).astype(int)
display(df)
print(df.shape)

tokens = sorted(set(c[0] for c in df.columns))
datasets = sorted(set(c[1] for c in df.columns))
print('Tokens:\n', tokens)
print('\nDatasets:\n', datasets)

,!=,%,&,(,),*,*=,+=,",",-,...,struct,switch,typedef,union,unsigned,void,wchar_t,while,{,}
,NTR,NTR,NTR,NTR,NTR,NTR,NTR,NTR,NTR,NTR,...,TBO,TBO,TBO,TBO,TBO,TBO,TBO,TBO,TBO,TBO
num,,,,,,,,,,,,,,,,,,,,,
0.0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.0001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.9990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.9992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(8129, 789)
Tokens:
 ['!=', '%', '%=', '&', '&=', '(', ')', '*', '*=', '+', '++', '+=', ',', '-', '--', '-=', '->', '->*', '.', '.*', '...', '/', '/=', ':', '::', ';', '<', '<<=', '<=', '=', '==', '>', '>=', '>>=', '?', 'AndAnd', 'BooleanLiteral', 'CharacterLiteral', 'Directive', 'EOF', 'FloatingLiteral', 'Identifier', 'IntegerLiteral', 'MultiLineMacro', 'Not', 'OrOr', 'PointerLiteral', 'StringLiteral', 'UserDefinedLiteral', '[', ']', '^', '^=', 'alignas', 'alignof', 'asm', 'auto', 'bool', 'break', 'case', 'catch', 'char', 'char16_t', 'char32_t', 'class', 'const', 'const_cast', 'constexpr', 'continue', 'decltype', 'default', 'delete', 'do', 'double', 'dynamic_cast', 'else', 'enum', 'explicit', 'export', 'extern', 'final', 'float', 'for', 'friend', 'goto', 'if', 'inline', 'int', 'long', 'mutable', 'namespace', 'new', 'noexcept', 'operator', 'override', 'private', 'protected', 'public', 'register', 'reinterpret_cast', 'return', 'short', 'signed', 'sizeof', 'static', 'static_assert', 'sta

In [ ]:
set(c[1] for c in df.columns)

big  = df.xs('BIG', axis=1, level=1)
wild = df.xs('WILD', axis=1, level=1)

big /= big.sum(axis=0)
wild /= wild.sum(axis=0)

diff = (big - wild) / wild
diff.dropna(how='all').style.background_gradient()

/opt/conda/lib/python3.8/site-packages/pandas/io/formats/style.py:1268: RuntimeWarning: invalid value encountered in double_scalars
  norm = colors.Normalize(smin - (rng * low), smax + (rng * high))
/opt/conda/lib/python3.8/site-packages/pandas/io/formats/style.py:1264: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/opt/conda/lib/python3.8/site-packages/pandas/io/formats/style.py:1265: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


In [3]:
from scipy.stats import ks_2samp
from IPython.display import display

GREEN = '#81C784'
RED = '#E57373'

def get_samples(col):
    sample = []
    for val, count in zip(col.index, col):
        try:
            sample += [ val ] * count
        except Exception as e:
            print(val, count)
            raise e
    return sample

def pvalue_reject(val):
    if np.isnan(val): return ''
    return f'background: {RED if val < 0.05 else GREEN}'

ks = pd.DataFrame()

for token in tqdm(tokens):
    df_token = df[token]
    samples = { col: get_samples(df_token[col]) for col in df_token.columns }
    
    for i, ds in enumerate(datasets):
        if ds not in samples or ds == 'WILD' or not samples[ds]: continue
        result = ks_2samp(samples['WILD'], samples[ds])
        ks.at[token, ds + '-p'] = result.pvalue
        ks.at[token, ds + '-s'] = result.statistic


ks = ks.sort_index(axis=0).sort_index(axis=1)
display((ks > 0.05).sum(axis=0))
display(ks.style.applymap(pvalue_reject))

BIG-p      1
BIG-s    122
CVS-p      0
CVS-s    105
JLT-p      1
JLT-s     87
MNY-p      0
MNY-s    112
NTR-p      5
NTR-s     40
TBO-p      0
TBO-s     56
VDC-p      0
VDC-s    127
dtype: int64

,BIG-p,BIG-s,CVS-p,CVS-s,JLT-p,JLT-s,MNY-p,MNY-s,NTR-p,NTR-s,TBO-p,TBO-s,VDC-p,VDC-s
!=,0.000000,0.517122,0.000000,0.233577,0.000000,0.521021,0.000000,0.089630,0.000001,0.881240,0.000000,0.947685,0.000000,0.608578
%,0.000000,0.571542,0.000000,0.240630,0.000000,0.882329,0.000000,0.117068,0.000000,0.927837,0.000000,0.987486,0.000000,0.802441
%=,0.000000,0.798806,0.000000,0.264118,nan,nan,0.000000,0.128613,nan,nan,nan,nan,0.000000,0.884168
&,0.000000,0.277471,0.000000,0.254263,0.000000,0.249964,0.000000,0.041041,0.000000,0.558630,0.009511,0.393833,0.000000,0.175690
&=,0.000000,0.589741,0.000000,0.327641,nan,nan,0.000000,0.192889,nan,nan,nan,nan,0.000000,0.763544
(,0.000000,0.231727,0.000000,0.206646,0.000000,0.311967,0.000000,0.054570,0.000658,0.250262,0.000000,0.685607,0.000000,0.179639
),0.000000,0.231400,0.000000,0.206627,0.000000,0.311958,0.000000,0.060153,0.000658,0.250258,0.000000,0.685632,0.000000,0.179640
*,0.000000,0.218761,0.000000,0.152355,0.000000,0.162207,0.000000,0.093728,0.000159,0.614458,0.000000,0.285680,0.000000,0.189566
*=,0.000000,0.585230,0.000000,0.392911,nan,nan,0.000000,0.215566,0.005399,0.948043,nan,nan,0.000000,0.846765
+,0.000000,0.366013,0.000000,0.116670,0.000000,0.349430,0.000000,0.028424,nan,nan,0.000000,0.906362,0.000000,0.460433


In [4]:
stats = ks.loc[:, ks.columns.str.endswith('-s')].copy()
stats['avg'] = stats.mean(axis=1)
stats.sort_values(by='avg').style.background_gradient()

,BIG-s,CVS-s,JLT-s,MNY-s,NTR-s,TBO-s,VDC-s,avg
=,0.099094,0.125159,0.094487,0.147843,0.210778,0.426160,0.062026,0.166507
->,0.158048,0.198709,0.362718,0.152766,nan,nan,0.203922,0.215232
*,0.218761,0.152355,0.162207,0.093728,0.614458,0.285680,0.189566,0.245251
(,0.231727,0.206646,0.311967,0.054570,0.250262,0.685607,0.179639,0.274345
),0.231400,0.206627,0.311958,0.060153,0.250258,0.685632,0.179640,0.275095
const,0.475048,0.127104,0.268931,0.066188,nan,nan,0.441973,0.275849
&,0.277471,0.254263,0.249964,0.041041,0.558630,0.393833,0.175690,0.278699
IntegerLiteral,0.079649,0.202523,0.161652,0.194589,0.495150,0.784725,0.042242,0.280076
Directive,0.095133,0.086667,0.718589,0.064444,0.296558,0.534465,0.223649,0.288501
},0.122307,0.110549,0.583182,0.177541,0.458058,0.501709,0.081317,0.290666


In [5]:
stats.describe()

,BIG-s,CVS-s,JLT-s,MNY-s,NTR-s,TBO-s,VDC-s,avg
count,122.000000,107.000000,87.000000,119.000000,40.000000,56.000000,128.000000,128.000000
mean,0.476226,0.282760,0.573011,0.206402,0.658117,0.831973,0.560779,0.484721
std,0.185222,0.185409,0.237330,0.163611,0.241491,0.192247,0.229476,0.121481
min,0.079649,0.028215,0.094487,0.028424,0.210778,0.285680,0.042242,0.166507
25%,0.345937,0.160051,0.382446,0.089697,0.485882,0.748785,0.423488,0.414378
50%,0.493196,0.246974,0.594700,0.161715,0.641596,0.921411,0.602704,0.492353
75%,0.603544,0.341125,0.763161,0.238450,0.889722,0.972292,0.730318,0.558958
max,0.946132,0.998756,0.964701,0.804550,0.997324,0.998535,0.973490,0.813341


In [6]:
# colors = plt.get_cmap('tab10').colors
# n_cols = 3
# n_rows = np.ceil(len(tokens) / n_cols).astype(int)
# size = 4
# fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(n_cols * size * 2, n_rows * size))

# for token, ax in tqdm(zip(tokens, axes.ravel()), total=len(tokens)):
#     df_token = df[token]
#     legend = []

#     for i, dataset in enumerate(['BIG', 'WILD']):#datasets):
#         if dataset not in df_token.columns: continue

#         legend.append(dataset)
#         col = df_token[dataset].fillna(0).replace([-np.inf, np.inf], 0)
#         try:
#             sns.kdeplot(x=col.index, weights=col, ax=ax, color=colors[i] if dataset != 'WILD' else 'red', linestyle='dotted' if dataset != 'WILD' else 'solid')
#         except:
#             pass

#     ax.set_xlim(0, 0.2)
#     ax.legend(legend, loc='upper right')
#     ax.set_title(token, fontweight='bold')

# plt.tight_layout()
# # plt.savefig('img/byfile-type-kde.svg')
# # plt.savefig('img/byfile-type-kde.jpg')
# plt.show()
# plt.close()

In [7]:
import ipywidgets as widgets
from IPython.display import display

datasets_widget = widgets.SelectMultiple(
    options=datasets,
    rows=8,
    value=[ 'BIG', 'WILD' ],
    description='Datasets',
)
token_widget = widgets.Dropdown(
    options=tokens,
    value='+',
    description='Token:',
)
max_xlim_widget = widgets.FloatText(
    options=tokens,
    value=0.2,
    description='xlim:',
)

def display_plot(selected, token, max_xlim):
    colors = plt.get_cmap('tab10').colors
    
    fig = plt.figure(figsize=(8, 6))
    ax = fig.gca()

    df_token = df[token]
    legend = []

    for i, dataset in enumerate(selected):
        if dataset not in df_token.columns: continue
        legend.append(dataset + (f': {ks.at[token, f"{dataset}-s"]:.3f}' if dataset != 'WILD' else ''))
        col = df_token[dataset].fillna(0).replace([-np.inf, np.inf], 0)
        try:
            sns.kdeplot(x=col.index, weights=col, ax=ax, color=colors[i] if dataset != 'WILD' else 'red', linestyle='dotted' if dataset != 'WILD' else 'solid')
        except:
            pass

        ax.set_xlim(0, max_xlim)
        ax.legend(legend, loc='upper right')
        ax.set_title(token, fontweight='bold')

    plt.tight_layout()
    plt.show()
    plt.close()

sidebar = widgets.VBox([datasets_widget, token_widget, max_xlim_widget])
out = widgets.interactive_output(
    display_plot,
    dict(
        selected = datasets_widget,
        token = token_widget,
        max_xlim = max_xlim_widget,
    )
)

ui = widgets.HBox([sidebar, out])

display(ui)